In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import json
from tqdm import tqdm_notebook

In [3]:
data = pd.read_csv('data/base/stories_description.csv', encoding="utf8")

In [4]:
data.head()

,story_id,story_json
0,127,"{""guid"":""770a5bae-0e3f-4a6b-b924-bd87bd51a038""..."
1,254,"{""guid"":""64f4c9ef-647b-4e04-b4d4-02297e939388""..."
2,865,"{""guid"":""3482206b-d223-4aec-92ba-0150055cd68a""..."
3,1491,"{""guid"":""5f4a9215-01de-4777-b70f-a18899db8f1c""..."
4,598,"{""guid"":""ed8754bd-67be-4fa1-9289-5508d96f1fa4""..."


In [5]:
data.loc[2].story_json

'{"guid":"3482206b-d223-4aec-92ba-0150055cd68a","type":"root","properties":{"name":"Тинькофф Инвестиции 2.0","background":{"gradient":{"colors":[],"direction":"vertical"},"blur":false},"icon_background":{"image_url":"https://static2.tinkoff.ru/portfolio/stories/images/investicii_11-05_zahod.jpg","gradient":{"colors":[{"hex":"#000000","alpha":0},{"hex":"#000000","alpha":0.6}],"direction":"vertical"},"blur":false},"hide_page_status":false},"content":[{"guid":"d59e8f2b-f41c-4943-bfb4-9ecd9a14b4fa","type":"story-page","properties":{"topMask":false,"bottomMask":false,"flip_interval":10,"background":{"image_url":"https://static2.tinkoff.ru/portfolio/stories/images/investicii_11-05_zahod.jpg","gradient":{"colors":[{"hex":"#000000","alpha":0.6},{"hex":"#000000","alpha":0},{"hex":"#000000","alpha":0},{"hex":"#000000","alpha":0},{"hex":"#000000","alpha":0},{"hex":"#000000","alpha":0},{"hex":"#000000","alpha":0},{"hex":"#000000","alpha":0},{"hex":"#000000","alpha":0},{"hex":"#000000","alpha":0}],

In [6]:
obj = json.loads(data.loc[5].story_json.replace("\\\\", "\\"))

In [7]:
obj.keys()

dict_keys(['guid', 'type', 'description', 'properties', 'content'])

In [8]:
obj["guid"]

'c4dfc7e9-c523-4790-b8a7-f649f2119340'

In [9]:
obj["content"][0]["content"][0]["content"][0]["properties"]

{'orientation': 'horizontal',
 'topMask': False,
 'bottomMask': False,
 'background': {'gradient': {'colors': [], 'direction': 'vertical'},
  'blur': False},
 'hidden': 'false',
 'type': 'grouped',
 'padding': {'left': 16},
 'alignment': 'center'}

In [10]:
obj["content"][0]["content"][0]["content"][0]["content"][0]["properties"]

{'url': 'https://static2.tinkoff.ru/portfolio/stories/icons/skrit__icon.png',
 'alignment': 'leading',
 'size': {'height': {'value': 88, 'units': 'pixels'},
  'width': {'value': 88, 'units': 'pixels'}},
 'corner_radius': {'value': 50, 'units': 'percent'},
 'content_mode': 'fill',
 'alpha': 1,
 'background': {'gradient': {'colors': [{'hex': '#ffffff', 'alpha': 0.2}],
   'direction': 'vertical'},
  'blur': False}}

In [11]:
obj["guid"]

'c4dfc7e9-c523-4790-b8a7-f649f2119340'

In [12]:
features = {
    "trash": {},
    "story_text": {},
    "story_font_size": {},
    "story_font_type": {},
    "story_font_color": {},
    "story_background_colors": {},
    "story_background_alpha": {},
    "story_background_width": {},
    "story_background_height": {}
}
bad_rows = 0

def parse(columns, name=None):
    now = obj
    last = "trash"
    
    for column in columns:
        last = column
        try:
            now = now[column]
        except:
            now = ""
            break
    
    last = columns[-1]
    
    if name is not None:
        last = name
    
    if last not in features:
        features[last] = {}
    
    features[last][key] = now
    

obj = None            

for index, row in tqdm_notebook(data.iterrows()):
    try:
        obj = json.loads(row.story_json.replace("\\\\", "\\"))

        key = row.story_id
        
        parse(["description"])
        
        parse(["properties", "name"])
        
        # parse(["properties", "background", "gradient", "direction"])
        # same values
        
        parse(["properties", "icon_background", "gradient", "colors", 0, "hex"], "icon_hex_0")
        
        parse(["properties", "icon_background", "gradient", "colors", 1, "hex"], "icon_hex_1")
        
        parse(["properties", "icon_background", "gradient", "colors", 0, "alpha"], "icon_alpha_0")
        
        parse(["properties", "icon_background", "gradient", "colors", 1, "alpha"], "icon_alpha_0")
        
        parse(["properties", "hide_page_status"], "hide_page_status")
        
        for i in range(10):
            for j in range(10):
                for e in range(10):
                    for r in range(10):
                        try:
                            props = obj["content"][i]["content"][j]["content"][e]["content"][r]["properties"]

                            try:
                                text = props["text"]
                                if key not in features["story_text"]:
                                    features["story_text"][key] = []
                                
                                features["story_text"][key].append(text)
                           
                                story_font_size = props["font_size"]
                                if key not in features["story_font_size"]:
                                    features["story_font_size"][key] = []
                                
                                features["story_font_size"][key].append(story_font_size)
                            
                                story_font_size = props["font_type"]
                                if key not in features["story_font_type"]:
                                    features["story_font_type"][key] = []
                                
                                features["story_font_type"][key].append(story_font_size)
                           
                                story_font_size = props["font_color"]["hex"]
                                if key not in features["story_font_color"]:
                                    features["story_font_color"][key] = []
                                
                                features["story_font_color"][key].append(story_font_size)
                            except:
                                pass
                            
                            try:
                                story_font_size = props["background"]["gradient"]["colors"][0]["hex"]
                                
                                if key not in features["story_background_colors"]:
                                    features["story_background_colors"][key] = []
                                
                                features["story_background_colors"][key].append(story_font_size)
                           
                                story_font_size = props["size"]["width"]["value"]
                                
                                if key not in features["story_background_width"]:
                                    features["story_background_width"][key] = []
                                
                                features["story_background_width"][key].append(story_font_size)
                            
                                story_font_size = props["size"]["height"]["value"]
                                
                                if key not in features["story_background_height"]:
                                    features["story_background_height"][key] = []
                                
                                features["story_background_height"][key].append(story_font_size)
                                
                                story_font_size = props["background"]["gradient"]["colors"][0]["alpha"]
                                
                                if key not in features["story_background_alpha"]:
                                    features["story_background_alpha"][key] = []
                                
                                features["story_background_alpha"][key].append(story_font_size)
                            except:
                                pass
                            
                        except:
                            pass
            
    except:
        raise
        bad_rows+=1
        pass

In [13]:
bad_rows

0

In [14]:
features.keys()

dict_keys(['trash', 'story_text', 'story_font_size', 'story_font_type', 'story_font_color', 'story_background_colors', 'story_background_alpha', 'story_background_width', 'story_background_height', 'description', 'name', 'icon_hex_0', 'icon_hex_1', 'icon_alpha_0', 'hide_page_status'])

In [15]:
features["story_background_alpha"][598]

[1, 1, 1, 1, 1]

In [16]:
result = pd.DataFrame(features)

In [17]:
result = result.reset_index().rename(columns={"index": "story_id"}).drop(columns="trash")

In [18]:
result.head()

,story_id,story_text,story_font_size,story_font_type,story_font_color,story_background_colors,story_background_alpha,story_background_width,story_background_height,description,name,icon_hex_0,icon_hex_1,icon_alpha_0,hide_page_status
0,127,[Как изменить ПИН-код],[36],[bold],[#ffffff],[#ffffff],[0.2],[88],[88],https://wiki.tcsbank.ru/pages/viewpage.action?...,Изменить пин-код,#b8a1e0,#00bee0,1,False
1,254,"[Как пополнить карту «Стрелка», В приложении Т...","[32, 20]","[bold, normal]","[#ffffff, #ffffff]",[#ffffff],[0.2],[88],[88],https://wiki.tcsbank.ru/pages/viewpage.action?...,Пополнить «Стрелку»,#b8a1e0,#00bee0,1,False
2,865,"[Тинькофф Инвестиции 2.0, У нас хорошие новост...","[36, 17, 22, 17, 22, 17, 22, 17, 22, 17, 22, 1...","[bold, normal, bold, normal, bold, normal, bol...","[#ffffff, #ffffff, #ffffff, #ffffff, #ffffff, ...",[#ffdd2c],[1],[0],[36],,Тинькофф Инвестиции 2.0,#000000,#000000,0.6,False
3,1491,"[Выставка уличной культуры, 18 и 19 августа пр...","[36, 17, 22, 17, 22, 17, 22, 17, 22, 17, 22, 1...","[bold, normal, bold, normal, bold, normal, bol...","[#ffffff, #ffffff, #ffffff, #ffffff, #ffffff, ...",NaN,NaN,NaN,NaN,,Что делать на Faces & Laces,#000000,#000000,0.6,False
4,598,"[Необычные виды спорта, Если бег, бассейн и тр...","[30, 15, 22, 17, 15, 22, 17, 15, 22, 17, 15, 2...","[bold, normal, bold, normal, normal, bold, nor...","[#ffffff, #ffffff, #ffffff, #ffffff, #000000, ...","[#ffdd2c, #ffdd2c, #ffdd2c, #ffdd2c, #ffdd2c]","[1, 1, 1, 1, 1]","[0, 0, 0, 0, 0]","[36, 36, 36, 36, 36]",,Необычные виды спорта,#000000,#000000,0.6,False


In [19]:
result.to_csv("data/base/stories_descriptions_parsed.csv", index=False)

In [20]:
result["icon_hex_0"].unique()

array(['#b8a1e0', '#000000', '', '#17d15f', '#20419a', '#ff7047'],
      dtype=object)